In [1]:
import numpy as np
import pandas as pd
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
import preliz as pz
az.style.use("arviz-grayscale")

In [2]:
bikes = pd.read_csv("https://raw.githubusercontent.com/aloctavodia/BAP3/main/code/data/bikes.csv")
y = bikes["rented"].values
X = bikes["hour"].values[:,None]

In [3]:
def get_ig_params(x_vals, l_b=None, u_b=None, mass=0.95, plot=False):
    """
    Returns a weakly informative prior for the length-scale parameter of the GP kernel.
    """

    differences = np.abs(np.subtract.outer(x_vals, x_vals))
    if l_b is None:
        l_b = np.min(differences[differences != 0]) * 2
    if u_b is None:
        u_b = np.max(differences) / 1.5

    dist = pz.InverseGamma()
    pz.maxent(dist, l_b, u_b, mass, plot=plot)

    return dict(zip(dist.param_names, dist.params))

In [4]:
ivg_prior = get_ig_params(X)
ivg_prior["alpha"]

/home/harrison/anaconda3/envs/pymc_env/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:2069: RuntimeWarning: divide by zero encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)


5.054174140477293

In [8]:
with pm.Model() as model_hsgp:
    ℓ = pm.InverseGamma('ℓ',alpha = ivg_prior["alpha"], beta = ivg_prior["beta"] )

    cov = pm.gp.cov.ExpQuad(1, ls=ℓ)
    gp = pm.gp.HSGP(m=[10], c=1.5, cov_func=cov)

    f = gp.prior('f', X=X)
    α = pm.HalfNormal('α', 1)
    _ = pm.NegativeBinomial("obs", alpha=α, mu= np.exp(f), observed=y)

    idata_hsgp = pm.sample(random_seed=123)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [ℓ, f_hsgp_coeffs_, α]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 93 seconds.


In [24]:
az.summary(idata_hsgp, var_names=['f_hsgp_coeffs_',"ℓ", "α"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
f_hsgp_coeffs_[0],8.796,0.301,8.208,9.336,0.008,0.006,1412.0,1822.0,1.0
f_hsgp_coeffs_[1],-1.335,0.174,-1.679,-1.024,0.005,0.003,1357.0,1451.0,1.0
f_hsgp_coeffs_[2],1.213,0.208,0.834,1.614,0.005,0.004,1559.0,1885.0,1.0
f_hsgp_coeffs_[3],0.265,0.295,-0.320,0.793,0.008,0.006,1380.0,1528.0,1.0
f_hsgp_coeffs_[4],1.263,0.395,0.485,1.958,0.011,0.008,1387.0,1991.0,1.0
f_hsgp_coeffs_[5],1.516,0.576,0.461,2.633,0.016,0.012,1226.0,1595.0,1.0
f_hsgp_coeffs_[6],0.844,0.636,-0.341,2.045,0.018,0.013,1309.0,1749.0,1.0
f_hsgp_coeffs_[7],-1.737,0.614,-2.920,-0.604,0.014,0.010,1899.0,2272.0,1.0
f_hsgp_coeffs_[8],-2.582,0.687,-3.900,-1.317,0.014,0.010,2325.0,2130.0,1.0
f_hsgp_coeffs_[9],-1.510,0.831,-3.076,0.032,0.019,0.013,1913.0,2380.0,1.0


In [20]:
idata_hsgp


Inference data with groups:
	> posterior
	> sample_stats
	> observed_data